<a href="https://colab.research.google.com/github/Dillonreed/3833Coursework/blob/main/3833Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wolrdwide Wellbeing Data Visualisation

---



## Imports/Data Preparation

In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

# Import wellbeing dataset and world map data
oecdData = pd.read_csv("https://raw.githubusercontent.com/Dillonreed/3833Coursework/main/OECD_betterLifeIndexWrangled.csv", encoding = "latin-1", delimiter=",")
worldMapSource = alt.topo_feature(data.world_110m.url, "countries")

## Chart Creation

### Interactivity Setup

In [3]:
# Create selections
countrySelect = alt.selection_multi(
    name="countrySelect",
    fields=["Country"]
)

countrySelectLegend = alt.selection_multi(
    name = "countrySelectLegend",
    fields=["Country"],
    bind = "legend"
)

# Create colours that are conditional on the selections
lifeSatisfactionColour = alt.condition(
    countrySelect | countrySelectLegend,
    alt.Color(
        "Life Satisfaction - Life satisfaction (Average Score):Q",
        scale=alt.Scale(scheme='plasma'),
        title = "Life satisfaction (Average Score)"
    ),
    alt.value("darkgrey")
)

### Life Satisfaction Worldwide Choropleth Map

In [11]:
# Create a background grey filled world map
background = alt.Chart(worldMapSource).mark_geoshape(fill="grey")

# Encode the map with data from dataset
foreground = (
    alt.Chart(worldMapSource).mark_geoshape(
        stroke = "black",
        strokeWidth = 0.2
    )
    .encode(
        color = (lifeSatisfactionColour),
        tooltip = [
            "Country:N",
            alt.Tooltip(
                "Life Satisfaction - Life satisfaction (Average Score):Q",
                title = "Life satisfaction (Average Score)"
            )
        ]
    )
    # Transform the data to link the map with household net wealth data using country codes
    .transform_lookup(
        lookup = "id",
        from_ = alt.LookupData(oecdData, "Country-Code", ["Life Satisfaction - Life satisfaction (Average Score)"] + ["Country"]),
    )
)

# Layer the background and foreground charts
lifeSatisfactionWorldWide = (
    (background + foreground)
    .properties(
        width= 850,
        height= 415,
        title = alt.TitleParams(
            "Worldwide Choropleth Map Showing The North-South Wellbeing Divide",
            subtitle = "Dark grey countries we have no data for"
            )
    )
    .add_selection(
        countrySelect, countrySelectLegend
    )
    .project("equalEarth")
)

lifeSatisfactionWorldWide

alt.LayerChart(...)

### Household Net Income by Country Bar Chart

In [5]:
# Set up a bar chart of Life Satisfaction by Country
householdNetWealthByCountry = alt.Chart(oecdData).mark_bar().encode(
    x = alt.X(
        "Country",
        scale = alt.Scale(zero = False),
        axis=alt.Axis(labelAngle=-45),
        sort="-y"
        ),

    y = alt.Y(
        "Income - Household net wealth (US Dollars)",
        scale = alt.Scale(zero = False),
        title = "Household net wealth (US Dollars)"
        ),

    # Display country names and exact values for the axes on hover for better identification
    tooltip=[
        "Country:N",
        alt.Tooltip(
            "Income - Household net wealth (US Dollars):Q",
            title = "Household net wealth (US Dollars)"
        )
    ],

    # Setting Colour
    color = lifeSatisfactionColour

).properties(
    title = "Household Net Wealth by Country",
    width = 849,
    height = 230
).add_selection(
    countrySelect, countrySelectLegend
)

householdNetWealthByCountry

alt.Chart(...)

### Quality of Support Network VS Homicide Rate Scatter Plot

In [6]:
# Set up a scatter plot displaying the Quality of support network against the Homicide rate of the different Countries
scatter2Base = alt.Chart(oecdData).mark_point().encode(
    y = alt.Y(
        "Community - Quality of support network (Percentage)",
        scale=alt.Scale(zero=False),
        title = "Quality of Support Network (Percentage)"
        ),

    x = alt.X(
        "Safety - Homicide rate (Ratio)",
        scale=alt.Scale(zero=False),
        title = "Homicide Rate (Ratio)"
        )
)

# Encode data points
scatter2EncodeData = scatter2Base.encode(
    # Display country names on hover for better identification
    tooltip = [
        "Country",
        alt.Tooltip(
            "Community - Quality of support network (Percentage)",
            title = "Quality of support network (Percentage)"
        ),
        alt.Tooltip(
            "Safety - Homicide rate (Ratio)",
            title = "Homicide rate (Ratio)"
        )
    ],
    # Set colour/shape for data points
    shape = "Country",
    color = lifeSatisfactionColour
)

# Compute and add regression line
scatter2RegressionLine = scatter2Base.transform_regression(
    "Community - Quality of support network (Percentage)",
    "Safety - Homicide rate (Ratio)"
).mark_line(color = 'black')

# Layer the scatter plot and regression line
qualityOfSupportNetworkVSHomicideRate = ((scatter2EncodeData + scatter2RegressionLine)
    .properties(
        title = alt.TitleParams(
            [
            "Scatter Graph Showing Negative Correlation",
            "between Quality of Support Network and Homicide Rate"
            ],

            subtitle = "Move around the visualization with mouse click/drag or scroll to zoom"
        ),
        width = 350,
        height = 325
    ).add_selection(
        countrySelect, countrySelectLegend
    ).interactive()
)

qualityOfSupportNetworkVSHomicideRate

alt.LayerChart(...)

### Employment Rate VS Labour Market Insecurity Scatter Plot

In [7]:
# Base scatter plot so data points and regression line can be coloured seperately
scatter1Base = alt.Chart(oecdData).mark_point().encode(
    x = alt.X(
        "Jobs - Labour market insecurity (Percentage)",
        scale=alt.Scale(zero=False),
        title = "Labour Market Insecurity (Percentage)"
        ),

    y = alt.Y(
        "Jobs - Employment rate (Percentage)",
        scale=alt.Scale(zero=False),
        title = "Employment Rate (Percentage)"
        )
)

# Encode data points
scatter1DataEncode = scatter1Base.encode(
    # Display country names and exact values for the axes on hover for better identification
    tooltip = [
        "Country:N",
        alt.Tooltip(
            "Jobs - Labour market insecurity (Percentage):Q",
            title = "Labour market insecurity (Percentage)"
        ),
        alt.Tooltip(
            "Jobs - Employment rate (Percentage):Q",
            title = "Employment rate (Percentage)"
        )
    ],

    # Set colour/shape for data points
    shape = "Country",
    color = lifeSatisfactionColour
)

# Compute and add regression line
scatter1RegressionLine = scatter1Base.transform_regression(
    "Jobs - Labour market insecurity (Percentage)",
    "Jobs - Employment rate (Percentage)"
).mark_line(color = "black")

# Layer the scatter plot and regression line
employmentRateVSLabourMarketInsecurity = ((scatter1DataEncode + scatter1RegressionLine)
    .properties(
        title = alt.TitleParams(
            [
            "Scatter Graph Showing Negative Correlation",
            "between Employment Rate and Labour Market Insecurity"
            ],

            subtitle = "Move around the visualization with mouse click/drag or scroll to zoom"
        ),
        width = 350,
        height = 325
    )
    .add_selection(
        countrySelect, countrySelectLegend
    )
    .interactive()
)

employmentRateVSLabourMarketInsecurity

alt.LayerChart(...)

## Multi-Coordinated View Setup

### Layout

In [8]:
# Create multi coordinated view for laying out of the data
# 2 X 2 Grid
multiCoordinatedView = alt.hconcat(
    alt.vconcat(lifeSatisfactionWorldWide, householdNetWealthByCountry),
    alt.vconcat(qualityOfSupportNetworkVSHomicideRate, employmentRateVSLabourMarketInsecurity),
    # Setup title with required text, and anchored in the middle
    title = alt.TitleParams(
        'Worldwide Wellbeing Dashboard',
        subtitle = ["Hover over any data to show exact values",
                    "Click on a Country to highlight it on all of the charts (Hold shift to select multiple)"],
        anchor = "middle"
    ),

).configure_legend(
    # Configure the legend to have a limit of 999 so it won't cut off the countries
    symbolLimit = 999,

    # Make the stroke colour black to make the legend more legible
    symbolStrokeColor = "black"
)

### Saving as a HTML File

In [9]:
# Save MCV as HTML
multiCoordinatedView.save("WorldwideWellbeing.html")